Задание 1.

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [3]:
import pandas as pd

ratings=pd.read_csv('ml-latest-small/ratings.csv', sep=',')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
users = ratings.groupby('userId').agg({'timestamp': ['min', 'max'], 'movieId': 'count'})
users.head()

timestamp             movieId
               min         max   count
userId                                
1       1260759108  1260759205      20
2        835355395   835356246      76
3       1298861589  1298932787      51
4        949778714   949982274     204
5       1163373044  1163375145     100

In [5]:
users = users[users['movieId']['count'] > 100]
users.head()

timestamp             movieId
               min         max   count
userId                                
4        949778714   949982274     204
8       1154389340  1154474527     116
15       997937239  1469330735    1700
17      1127468587  1127476640     363
19       855190091   855195373     423

In [6]:
users['Lifetime'] = users.apply(lambda row: row['timestamp']['max'] - row['timestamp']['min'], axis=1)
users.head()

timestamp             movieId   Lifetime
               min         max   count           
userId                                           
4        949778714   949982274     204     203560
8       1154389340  1154474527     116      85187
15       997937239  1469330735    1700  471393496
17      1127468587  1127476640     363       8053
19       855190091   855195373     423       5282

In [9]:
average_lifetime = round(users['Lifetime'].mean())
print(average_lifetime)

40080507


Задание 2.

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [16]:
import numpy as np

In [12]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [19]:
datatable =rzd.merge(auto, how='outer', on='client_id').merge(air, how='outer', on='client_id')
datatable[np.isnan(datatable)] = 0
datatable

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [21]:
fulltable=datatable.merge(client_base, how='outer', on='client_id')
fulltable

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


Задача 3.

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? 

Для составления ответа можно использовать вопросы:
        У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
        Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
        Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Скорее всего это была бы отдельная таблица данных. 

Кроме широты и долготы также требуется значение timestamp каждого посещения и покупки для связи со временем снятия координат. Скорее всего не необходимости хранить детальную геопозицию до часов и минут широты и долготы, можно найти способ сбора наиболее посещаемых мест (приблизить данные) и присвоить им id, которые и войдут в таблицу визитов и покупок наравне со временем соверщения действия. Это как возможная задумка, не претендует на истину.

Информацию из этих объединенных данных можно вынести самую разнообразную: наиболее посещаемые пользователями места для размещения рекламы, время преимущественной активности пользователей для распространения пуш-уведомлений, кластеризация пользователей по предпочтениям, возрастным группам, интересам для продвижения наиболее удачных продуктов и т.д. 